全局变量：
        #生长：
        
        存储能量比率：energystorage=100％
        能量消耗速度：speed=1时，energyrate=/time   speed=2时，energyrate=/time  呈平方递推  （需要一个基准物种）
        饥饿比率：hungrynum=30％

生长：
        给生物自身设置一个时间表，记录其经历的刷新次数，通过刷新次数决定，每次从食物中扣除多少用以自身生长。
        大概24刷新的时间进行一次捕食
        100次刷新成长一岁
        正常年纪的30％为成年期
        前百分之30：energy=-500/(9*(t^2))*(x^2)+100/(3*t)*x+1
        后百分之70：energy=-500/(49*(t^2))*(x^2)+300/(49*t)*x+249/49
        假设成年体重是幼年体中的六倍
        幼年能量记为authenweight
        各年龄能量呈正态分布，方差为0.25authenweight
        
捕食：
        捕食对象的能量含量：preyenergy

运动：
        饥饿状态下的运动是先找到运动方向，再朝着这个方向一直运动
搜索：
        搜索所有该动物的列表，看是否有位置在感知范围内的动物
        无，返回0，0
        有，返回1，最近的该物种
记忆：
        确定是否是水源，在确定是否存在于记忆之中，若是水源且不存在记忆之中，就把该位置储存到memory_data里
捕猎：
        按照给定速度
        确定与对象的距离是否能在一回合内接近
        刚开始消耗耐力值进行高速奔跑
        耐力值耗尽后以正常速度奔跑
        捕食成功返回1，未成功返回0
耐力值系统：
        每time回复5，高速消耗25
        耐力值归零后，耐力值判定为0，动物不能继续奔跑
        等到耐力值回到最大值后，耐力值判定为1，动物可以选择继续奔跑
逃跑过程：
        按照给定速度
        确定与对象的距离是否能在一回合内接近
        刚开始消耗耐力值进行高速奔跑
        耐力值耗尽后以正常速度奔跑
        被捕食成功返回1，未成功返回0
渴系统：
        渴度为100
        每time消耗1
觅水行为：
        以最近距离前往记忆中的水位
        先确定去哪个水位，并返回该水位
        在行动
求偶行为：
        首先搜索对象
        对象性别必须为异性
        沿用饥饿的方向函数
        找到对象之后，二者向对方的位置前进
        到达同一位置则返回1和最远位置坐标，否则返回0和最远位置坐标

In [239]:
import numpy as np
import math

In [297]:
class biology:
    def __init__(self,speed,visibility,age,ageaverage,authenweight,energyweight,energystorage,setting,food,enermy,bio):#分别代表生物的运动能力，感知范围，年龄,寿命,幼年体重,
        #，初始体重，初始储存能量,初始位置，食物,天敌,族群
        self.no_is=1  #存活判定
        self.speed=speed
        self.visibility=visibility
        self.age=float(age)
        self.ageaverage=ageaverage
        self.authenweight=authenweight
        self.energyweight=energyweight
        self.energystorage=energystorage
        self.setting=setting
        self.food=food
        self.enermy=enermy
        self.memory_data=[]
        self.thirs=100#渴值
        self.hungry_is=0#是否饥饿
        self.seekis=0#是否开始寻找
        self.enermy_is=0#周围是否有天敌
        self.enermy_mem=0#最近的天敌
        self.food_is=0#周围是否有食物
        self.food_mem=0#最近的食物
        self.energy_lose=0#每个time损失的能量
        self.bear=100#耐力值
        self.partner_is=0#发情期判定
        self.part_is=0#周围是否有配偶目标
        self.part_mem=0#最近的配偶目标
        self.baby_is=0#怀孕判定
        self.baby_time=0#怀孕时间
        self.bio=bio

        self.size = 5

    def draw(self, gameDisplay):
        # global rabbit
        # global wolf
        # if self.bio == rabbit:
        #     draw.circle(gameDisplay, (0, 0, 0), self.setting, self.size)
        # if self.bio == wolf:
        draw.circle(gameDisplay, (0, 255, 0), self.setting, self.size)


    def timemaker(self):
        self.time=self.age*100
        self.time=self.time+1
        self.age=self.time/100
        return 1
        

    def age_energy_maker(self):#生长过程
        if self.age<=self.ageaverage*0.3:
            weightaverage=(-500/(9*(self.ageaverage**2))*(self.age**2)+100/(3*self.ageaverage)*self.age+1)*self.authenweight
        elif (self.age>self.ageaverage*0.3) & (self.age<self.ageaverage):
            weightaverage=(-500/(49*(self.ageaverage**2))*(self.age**2)+300/(49*self.ageaverage)*self.age+249/49)*self.authenweight
        else:
            self.no_is=0
            weightaverage=self.authenweight
        result = np.random.normal(weightaverage, 0.25*self.authenweight, 1)
        a=self.energyweight
        self.energyweight=result[0]
        self.energystorage=self.energystorage-(self.energyweight-a)
        if self.energystorage==0:
            self.no_is=0

    def walk(self):#饱食状态下的运动过程
        b=[]
        for i in range(self.setting[0]-self.speed,self.setting[0]+self.speed+1):
            for j in range(self.setting[1]-self.speed,self.setting[1]+self.speed+1):
                a=((i-self.setting[0])**2+(j-self.setting[1])**2)**(0.5)
                if a<=self.speed:
                    b.append([i,j])
        c=np.random.randint(0,len(b))
        if b[c][0]<0:
            b[c][0]+=800
        elif b[c][1]<0:
            b[c][1]+=800
        self.setting=b[c]
        self.energy_lose+=self.speed
        self.energystorage-=self.speed

    def seekfor(self):#刚开始饥饿运动时，要先确定寻找的方向
        pass

    def seek(self):#饥饿状态下的觅食运动
        if (self.hungry==1) & (self.seekis==0):
            self.x,self.y=self.seekfor()
            seekis=1
        self.setting[0]+=self.x
        self.setting[1]+=self.y

    def see(self,cate):#搜索函数
        b=[]
        no=biology(0,0,0,0,0,0,0,[0,0],'none','none','none')
        for i in range(self.setting[0]-self.visibility,self.setting[0]+self.visibility+1):
            for j in range(self.setting[1]-self.visibility,self.setting[1]+self.visibility+1):
                a=((i-self.setting[0])**2+(j-self.setting[1])**2)**(0.5)
                if a<=self.visibility:
                    b.append([i,j])
        c=[]
        for m in cate:
            for n in b:
                if (m.setting==n)&(m!=self):
                    c.append(m)
        if c==[]:
            return 0,no
        else:
            d=c[0]
            for i in c:
                x=((i.setting[0]-self.setting[0])**2+(i.setting[1]-self.setting[1])**2)**0.5
                y=((d.setting[0]-self.setting[0])**2+(d.setting[1]-self.setting[1])**2)**0.5
                if x<=y:
                    d=i
            return 1,d

    def seeforpart_mem(self,cate):#搜索对象的函数
        b=[]
        no=biology(0,0,0,0,0,0,0,[0,0],'none','none','none')
        for i in range(self.setting[0]-self.visibility,self.setting[0]+self.visibility+1):
            for j in range(self.setting[1]-self.visibility,self.setting[1]+self.visibility+1):
                a=((i-self.setting[0])**2+(j-self.setting[1])**2)**(0.5)
                if a<=self.visibility:
                    b.append([i,j])
        c=[]
        for m in cate:
            for n in b:
                if (m.setting==n)&(m!=self)&(m.partner_is==1):
                    c.append(m)
        if c==[]:
            return 0,no
        else:
            d=c[0]
            for i in c:
                x=((i.setting[0]-self.setting[0])**2+(i.setting[1]-self.setting[1])**2)**0.5
                y=((d.setting[0]-self.setting[0])**2+(d.setting[1]-self.setting[1])**2)**0.5
                if x<=y:
                    d=i
            return 1,d

    def memory(self,water):
        for i in water:
            a=1
            for j in self.memory_data:
                if i==j:
                    a=0
            if (a==1)&(self.setting==i):
                self.memory_data.append(i)

    def bear_get(self):#耐力值获取系统
        self.bear+=5
        if self.bear>=100:
            self.bear=100
            self.quick_is=1

    def bear_dec(self):#耐力值消耗系统
        self.bear-=25
        if self.bear<=0:
            self.bear=0
            self.quick_is=0

    def predation(self,food,speed_s,speed_i):#捕食过程
        a=((food.setting[0]-self.setting[0])**2+(food.setting[1]-self.setting[1])**2)**0.5
        if a<=(speed_s-speed_i):
            self.setting=food.setting
            return 1
        else:
            for i in range(self.setting[0]-speed_s,self.setting[0]+speed_s+1):
                for j in range(self.setting[1]-speed_s,self.setting[1]+speed_s+1):
                    c=((i-self.setting[0])**2+(j-self.setting[1])**2)**(0.5)
                    if c<=speed_s:
                        b.append([i,j])
            n=b[0]
            for m in b:
                d=((m[0]-food.setting[0])**2+(m[1]-food.setting[1])**2)**(0.5)
                e=((n[0]-food.setting[0])**2+(n[1]-food.setting[1])**2)**(0.5)
                if d<=e:
                    n=m
            self.setting=n
            return 0

    def escape(self,enermy,speed_s,speed_i):#逃跑过程
        a=((enermy.setting[0]-self.setting[0])**2+(enermy.setting[1]-self.setting[1])**2)**0.5
        if (a<=(speed_i-speed_s))&(enermy.food_is==1)&(enermy.food_mem==self)&(enermy.hungry_is==1):
            enermy.energystorage+=self.energystorage
            enermy.energystorage+=self.energyweight
            enermy.energy_lose+=speed_i
            self.energyweight=0
            self.energystorage=0
            self.energy_lose+=speed_s
            return 0
        else:
            b=[]
            for i in range(self.setting[0]-speed_s,self.setting[0]+speed_s+1):
                for j in range(self.setting[1]-speed_s,self.setting[1]+speed_s+1):
                    c=((i-self.setting[0])**2+(j-self.setting[1])**2)**(0.5)
                    if c<=speed_s:
                        b.append([i,j])
            n=b[0]
            for m in b:
                d=((m[0]-enermy.setting[0])**2+(m[1]-enermy.setting[1])**2)**(0.5)
                e=((n[0]-enermy.setting[0])**2+(n[1]-enermy.setting[1])**2)**(0.5)
                if d>=e:
                    n=m
            self.setting=n
            self.energy_lose+=speed_s
            self.energystorage-=speed_s
            return 1

    
    def thirsty(self):#渴觉系统
        self.thirs-=1
    
    def seekforwater(self):#觅水行为,确定方向
        a=self.memory_data[0]
        for i in self.memory_data:
            m=((a[0]-self.setting[0])**2+(a[1]-self.setting[1])**2)**(0.5)
            n=((i[0]-self.setting[0])**2+(i[1]-self.setting[1])**2)**(0.5)
            if n<=m:
                a=i
        return a

    def runforwater(self,target):#开始觅水
        b=[]
        for i in range(self.setting[0]-self.speed,self.setting[0]+self.speed+1):
            for j in range(self.setting[1]-self.speed,self.setting[1]+self.speed+1):
                a=((i-self.setting[0])**2+(j-self.setting[1])**2)**(0.5)
                if a<=self.speed:
                    b.append([i,j])
        c=b[0]
        for i in b:
            m=((c[0]-target[0])**2+(c[1]-target[1])**2)**(0.5)
            n=((i[0]-target[0])**2+(i[1]-target[1])**2)**(0.5)
            if n<=m:
                c=i
        self.setting=c
        if c==target:
            self.thirs=100

    def courtship(self,target):#配偶寻觅机制
        x=((self.setting[0]-target.setting[0])**2+(self.setting[1]-target.setting[1])**2)**0.5
        b=[]
        for i in range(self.setting[0]-self.speed,self.setting[0]+self.speed+1):
            for j in range(self.setting[1]-self.speed,self.setting[1]+self.speed+1):
                a=((i-self.setting[0])**2+(j-self.setting[1])**2)**(0.5)
                if a<=self.speed:
                    b.append([i,j])
        c=b[0]
        for i in b:
            m=((c[0]-target[0])**2+(c[1]-target[1])**2)**(0.5)
            n=((i[0]-target[0])**2+(i[1]-target[1])**2)**(0.5)
            if n<=m:
                c=i
        if x<=self.speed+target.speed:
            return 1,c
        else:
            return 0,c

    def eatgrass(self,grass):
        self.energystorage+=grass[self.setting[0]][self.setting[1]]
        grass[self.setting[0]][self.setting[1]]=0

    def partnership(self):
        if (self.age>=self.ageaverage*0.2)&(self.age<=self.ageaverage*0.3)&(self.baby_is==0):
            self.partner_is=1
        else:
            self.partner_is=0

    def sex(self,target):
        if self.part_is==0:
            pass
        else:
            self.baby_is=1
            self.partner_is=0
            target.baby_is=1
            target.partner_is=0
        
    def babygrow(self):
        self.baby_time+=1
        if self.baby_time>=5*self.ageaverage:
            self.bio.append(biology(self.speed,self.visibility,0,self.ageaverage,self.authenweight,self.authenweight,0.5*self.energystorage,self.setting,self.food,self.enermy,self.bio))
            self.energyweight-=self.authenweight
            self.energyweight=self.energyweight*0.5
            self.baby_is=0

In [298]:
#调试用食草动物对象
wolf=[]
rabbit=[]
wolf_1=biology(2,4,1,10,5,100,100,[2,1],rabbit,'none',wolf)
rabbit_1=biology(3,5,2.5,10,5,100,100,[5,60],'grass',wolf,rabbit)#分别代表生物的运动能力，感知范围，年龄,寿命,幼年体重,初始体重，初始储存能量,初始位置，食物,天敌,族群
rabbit_2=biology(3,5,2.5,10,5,100,100,[5,59],'grass',wolf,rabbit)
rabbit_2.partner_is=1
wolf.append(wolf_1)
rabbit.append(rabbit_1)
rabbit.append(rabbit_2)
grass=[]#地块生产能量列表
for i in range(100):
    a=[]
    for j in range (100):
        a.append(100)
    grass.append(a)
water=[]#水源地块
for i in range(5,10):
    for j in range(10,20):
        water.append([i,j])
rabbit_1.memory_data=[[7,15]]#初始记忆地块

In [ ]:
#食肉系动物
#输出1：年龄改变+返回值 （年龄增长）
#输出2：体重，储存能量   （生长）
#输出3：是否有天敌  最近的天敌位置   （危险察觉）
#输出4.1：存活情况 能量消耗 能量储存  位置  记忆地块（察觉到危险后逃跑）
#输出4.2：位置   能量储存  能量消耗    记忆地块（没有危险，饥饿时的摄食行为）
#输出4.3：位置，目标，渴度
#输出4.4：是否有对象  对象位置  位置  是否怀孕（寻找配偶）
#输出5：怀孕时间
def neuron(creature):
    creature.energy_lose=0#每个time损失的能量归零
    creature.timemaker()#年龄增长
    creature.partnership()#发情期判定
    print(creature.age)
    creature.age_energy_maker()#生长
    print(creature.energyweight,creature.energystorage)
    if creature.enermy!='none':
        creature.enermy_is,creature.enermy_mem=creature.see(creature.enermy)#检索周围，是否有天敌
        print(creature.enermy_is,creature.enermy_mem.setting)
    if creature.enermy_is==1:#如果有天敌，开始逃跑
        if (creature.bear>=25)&(creature.enermy_mem.bear>=25):#如果双方都有耐力值剩余，两者以2倍速度展开追逐
            creature.no_is=creature.escape(creature.enermy_mem,2*creature.speed,2*creature.enermy_mem.speed)
            creature.bear_dec()
            creature.enermy_mem.bear_dec()
        elif (creature.bear<25)&(creature.enermy_mem.bear>=25):
            creature.no_is=creature.escape(creature.enermy_mem,creature.speed,2*creature.enermy_mem.speed)
            creature.enermy_mem.bear_dec()
        elif (creature.bear>=25)&(creature.enermy_mem.bear<25):
            creature.no_is=creature.escape(creature.enermy_mem,2*creature.speed,creature.enermy_mem.speed)
            creature.bear_dec()
        elif (rabbit_1.bear<25)&(rabbit_1.enermy_mem.bear<25):#如果双方都没有耐力值剩余，两者以最大速度展开追逐
            creature.no_is=creature.escape(creature.enermy_mem,creature.speed,creature.enermy_mem.speed)
        creature.memory(water)#记忆可能遇到的水源
        print(creature.no_is,creature.energy_lose,creature.energystorage,creature.setting,creature.memory_data)
    elif creature.energystorage<=50:#如果储存的能量不足，开始觅食运动
        creature.hungry_is=1
        creature.walk()#随机运动
        creature.memory(water)#记忆可能遇到的水源
        if creature.food=='grass':
            creature.eatgrass(grass)#获取地块上的全部能量
        else:
            creature.food_is,creature.food_mem=creature.see(creature.food)
            creature.predation(creature.food_mem,creature.speed,creature.food_mem.speed)
        print(creature.setting,creature.energystorage,creature.energy_lose,creature.memory_data)
    elif creature.thirs<=30:#如果水分不足，开始觅水
        target=creature.seekforwater()
        creature.runforwater(target)
        print(creature.setting,target,creature.thirs)
    elif creature.partner_is==1:#如果处于发情期，开始寻找配偶
        creature.walk()#随机运动
        creature.memory(water)#记忆可能遇到的水源
        creature.part_is,creature.part_mem=creature.seeforpart_mem(creature.bio)#寻找周围环境中可能存在的配偶
        creature.sex(creature.part_mem)
        print(creature.part_is,creature.part_mem.setting,creature.setting,creature.baby_is)
    else:#在没有任何行为目的的情况下，开始自由运动
        creature.walk()
        creature.memory(water)#记忆可能遇到的水源
    if creature.baby_is==1:#如果怀孕，孩子的年龄孕期增加
        creature.babygrow()
        print(creature.baby_time)
    creature.thirsty()#每time结束，扣除一点渴觉
    if creature.thirs<=0:
        creature.no_is=0
    if creature.energystorage<=0:
        creature.no_is=0
    if creature.no_is==0:
        creature.remove(creature)

In [ ]:
#群体
for i in range(20):
    a=np.random.randint(0,800)
    b=np.random.randint(0,800)
    rabbit.append(biology(3,5,2.5,10,5,100,100,[a,b],'grass',wolf,rabbit))
for j in range(10):
    a=np.random.randint(0,800)
    b=np.random.randint(0,800)
    wolf.append=biology(2,4,1,10,5,100,100,[a,b],rabbit,'none',wolf)
for creature in wolf:
    neuron(creature)
for creature in rabbit:
    neuron(creature)


